In [1]:
import onnx
from finn.util.basic import make_build_dir
from finn.util.visualization import showInNetron
from finn.util.test import get_test_model_trained
import brevitas.onnx as bo
from finn.core.modelwrapper import ModelWrapper
from finn.transformation.infer_shapes import InferShapes
from finn.transformation.fold_constants import FoldConstants
from finn.transformation.general import GiveReadableTensorNames, GiveUniqueNodeNames, RemoveStaticGraphInputs
from finn.util.pytorch import ToTensor
from finn.transformation.merge_onnx_models import MergeONNXModels
from finn.core.datatype import DataType

model = ModelWrapper("QVGG.onnx")
global_inp_name = model.graph.input[0].name
ishape = model.get_tensor_shape(global_inp_name)
# preprocessing: torchvision's ToTensor divides uint8 inputs by 255
totensor_pyt = ToTensor()
chkpt_preproc_name = "QVGG_preproc.onnx"
bo.export_finn_onnx(totensor_pyt, ishape, chkpt_preproc_name)

# join preprocessing and core model
pre_model = ModelWrapper(chkpt_preproc_name)
model = model.transform(MergeONNXModels(pre_model))
# add input quantization annotation: UINT8 for all BNN-PYNQ models
global_inp_name = model.graph.input[0].name
model.set_tensor_datatype(global_inp_name, DataType["UINT8"])

from finn.transformation.insert_topk import InsertTopK
from finn.transformation.infer_datatypes import InferDataTypes

# postprocessing: insert Top-1 node at the end
model = model.transform(InsertTopK(k=1))
chkpt_name = "QVGG_pre_post.onnx"
# tidy-up again
model = model.transform(InferShapes())
model = model.transform(FoldConstants())
model = model.transform(GiveUniqueNodeNames())
model = model.transform(GiveReadableTensorNames())
model = model.transform(InferDataTypes())
model = model.transform(RemoveStaticGraphInputs())
model.save(chkpt_name)

/workspace/finn-base/src/finn/transformation/infer_data_layouts.py:119: UserWarning: Assuming 4D input is NCHW
  warnings.warn("Assuming 4D input is NCHW")


In [2]:
showInNetron(chkpt_name)

Serving 'QVGG_pre_post.onnx' at http://0.0.0.0:8081


In [3]:
import estimator

In [4]:
estimator.earlyStagePhysicalCheck(MODEL_FILE=chkpt_name,MVAU_WWIDTH_MAX=80,TARGET_FPS=1000000)

Now reading model file QVGG_pre_post.onnx
Previous run results deleted!
Building dataflow accelerator from QVGG_pre_post.onnx
Intermediate outputs will be generated in /tmp/finn_dev_xilinx
Final outputs will be generated in estimation
Build log is at estimation/build_dataflow.log
Running step: step_qonnx_to_finn [1/8]
Running step: step_tidy_up [2/8]
Running step: step_streamline [3/8]
Running step: step_convert_to_hls [4/8]
Running step: step_create_dataflow_partition [5/8]
Running step: step_target_fps_parallelization [6/8]
Running step: step_apply_folding_config [7/8]
Running step: step_generate_estimate_reports [8/8]
Completed successfully

 estimation/report/estimate_network_performance.json 

critical_path_cycles : 50244
max_cycles : 7146
max_cycles_node_name : ConvolutionInputGenerator_0
estimated_throughput_fps : 13993.84270920795
estimated_latency_ns : 502440.0

 estimation/report/estimate_layer_resources.json 

total :
	 BRAM_18K : 152.0
	 LUT : 881251.0
	 URAM : 0.0
	 DSP : 

In [16]:
estimator.earlyStagePhysicalSim(FPGA_PART="xc7z020clg400-1",MODEL_FILE=chkpt_name,MVAU_WWIDTH_MAX=80,TARGET_FPS=1000000)

Now reading model file Qlenet__pre_post.onnx
Previous run results deleted!
Building dataflow accelerator from Qlenet__pre_post.onnx
Intermediate outputs will be generated in /tmp/finn_dev_xilinx
Final outputs will be generated in rtlsim_performance
Build log is at rtlsim_performance/build_dataflow.log
Running step: step_qonnx_to_finn [1/17]
Running step: step_tidy_up [2/17]
Running step: step_streamline [3/17]
Running step: step_convert_to_hls [4/17]
Running step: step_create_dataflow_partition [5/17]
Running step: step_target_fps_parallelization [6/17]
Running step: step_apply_folding_config [7/17]
Running step: step_generate_estimate_reports [8/17]
Running step: step_hls_codegen [9/17]
Running step: step_hls_ipgen [10/17]


Traceback (most recent call last):
  File "/workspace/finn/src/finn/builder/build_dataflow.py", line 166, in build_dataflow_cfg
    model = transform_step(model, cfg)
  File "/workspace/finn/src/finn/builder/build_dataflow_steps.py", line 426, in step_hls_ipgen
    model = model.transform(HLSSynthIP())
  File "/workspace/finn-base/src/finn/core/modelwrapper.py", line 141, in transform
    (transformed_model, model_was_changed) = transformation.apply(
  File "/workspace/finn-base/src/finn/transformation/base.py", line 105, in apply
    new_nodes_and_bool = p.map(self.applyNodeLocal, old_nodes, chunksize=1)
  File "/opt/conda/lib/python3.8/multiprocessing/pool.py", line 364, in map
    return self._map_async(func, iterable, mapstar, chunksize).get()
  File "/opt/conda/lib/python3.8/multiprocessing/pool.py", line 765, in get
    self.wait(timeout)
  File "/opt/conda/lib/python3.8/multiprocessing/pool.py", line 762, in wait
    self._event.wait(timeout)
  File "/opt/conda/lib/python3.8/thr

> /opt/conda/lib/python3.8/threading.py(302)wait()
    300         try:    # restore state no matter what (e.g., KeyboardInterrupt)
    301             if timeout is None:
--> 302                 waiter.acquire()
    303                 gotit = True
    304             else:

--KeyboardInterrupt--

KeyboardInterrupt: Interrupted by user
Build failed


FileNotFoundError: [Errno 2] No such file or directory: 'rtlsim_performance/time_per_step.json'

In [15]:
import onnx
import torch
import finn.builder.build_dataflow as build
import finn.builder.build_dataflow_config as build_cfg
import os
import shutil
import json
import ast
import pandas as pd
from finn.builder.build_dataflow_config import ShellFlowType

MODEL_FILE = chkpt_name
OUTPUT_DIR = "synthesis"
MVAU_WWIDTH_MAX=80
SYNTH_CLK_PERIOD_NS = 10.0
TARGET_FPS = 1000000
FPGA_PART = "xc7z020clg400"
        
cfg = build.DataflowBuildConfig(
output_dir          = OUTPUT_DIR,
mvau_wwidth_max     = MVAU_WWIDTH_MAX,
target_fps          = TARGET_FPS,
synth_clk_period_ns = SYNTH_CLK_PERIOD_NS,
board               = "Pynq-Z2",
shell_flow_type     = build_cfg.ShellFlowType.VIVADO_ZYNQ,
generate_outputs=[
    build_cfg.DataflowOutputType.BITFILE,
    build_cfg.DataflowOutputType.PYNQ_DRIVER,
    build_cfg.DataflowOutputType.DEPLOYMENT_PACKAGE,
    ]
)
build.build_dataflow_cfg(MODEL_FILE, cfg)



Building dataflow accelerator from Qlenet__pre_post.onnx
Intermediate outputs will be generated in /tmp/finn_dev_xilinx
Final outputs will be generated in synthesis
Build log is at synthesis/build_dataflow.log
Running step: step_qonnx_to_finn [1/17]
Running step: step_tidy_up [2/17]
Running step: step_streamline [3/17]
Running step: step_convert_to_hls [4/17]
Running step: step_create_dataflow_partition [5/17]
Running step: step_target_fps_parallelization [6/17]
Running step: step_apply_folding_config [7/17]
Running step: step_generate_estimate_reports [8/17]
Running step: step_hls_codegen [9/17]
Running step: step_hls_ipgen [10/17]
Running step: step_set_fifo_depths [11/17]
Running step: step_create_stitched_ip [12/17]
Running step: step_measure_rtlsim_performance [13/17]
Running step: step_out_of_context_synthesis [14/17]
Running step: step_synthesize_bitfile [15/17]
Running step: step_make_pynq_driver [16/17]
Running step: step_deployment_package [17/17]
Completed successfully


0